In [1]:
import yfinance as yf
import pandas as pd
import numpy as np
import talib
import matplotlib.pyplot as plt

# Task 1: Correlation Analysis
nasdaq_data = yf.download('^IXIC', start='2000-01-01', end='2023-06-11', progress=False)
nse_data = yf.download('^NSEI', start='2000-01-01', end='2023-06-11', progress=False)

nasdaq_close = nasdaq_data['Close']
nse_close = nse_data['Close']

correlation = nasdaq_close.corr(nse_close)
relationship = 'positive' if correlation > 0 else 'negative'
strength = abs(correlation)

print(f"The correlation coefficient between NASDAQ and NSE indices is {correlation:.2f}.")
print(f"The relationship is {relationship} with a strength of {strength:.2f}.\n")

# Task 2: Lead-Lag Relationship
nasdaq_returns = nasdaq_close.pct_change().dropna()
nse_returns = nse_close.pct_change().dropna()

nasdaq_lagged = nasdaq_returns.shift(1)
nse_lagged = nse_returns.shift(1)

nasdaq_lagged_aligned, nse_returns_aligned = nasdaq_lagged.align(nse_returns, join='inner')

lead_lag_relationship = 'NASDAQ leads NSE' if (nasdaq_lagged_aligned > nse_returns_aligned).all() else 'NSE leads NASDAQ'

print(f"The lead-lag relationship between the indices is: {lead_lag_relationship}.\n")

# Task 3: Indicator Coding
def calculate_keltner_channel(data, period=20, multiplier=2):
    typical_price = (data['High'] + data['Low'] + data['Close']) / 3
    keltner_mid = typical_price.rolling(window=period).mean()
    atr = data['High'] - data['Low']
    keltner_upper = keltner_mid + (multiplier * atr)
    keltner_lower = keltner_mid - (multiplier * atr)
    return keltner_upper, keltner_mid, keltner_lower

def calculate_bollinger_bands(data, period=20, multiplier=2):
    sma = data.rolling(window=period).mean()
    std = data.rolling(window=period).std()
    upper_band = sma + (multiplier * std)
    lower_band = sma - (multiplier * std)
    return upper_band, sma, lower_band

upper_keltner, middle_keltner, lower_keltner = calculate_keltner_channel(nasdaq_data, period=20)
upper_bollinger, middle_bollinger, lower_bollinger = calculate_bollinger_bands(nasdaq_close, period=20)

# Task 4: Parameter Optimization
def calculate_sharpe_ratio(returns):
    risk_free_rate = 0.0
    return (np.mean(returns) - risk_free_rate) / np.std(returns)

def evaluate_strategy(data, signal):
    position = pd.Series(np.where(signal > 0, 1, -1), index=data.index)
    returns = data.pct_change() * position.shift(1)
    returns = returns.dropna()
    sharpe_ratio = calculate_sharpe_ratio(returns)
    return sharpe_ratio

nasdaq_data['Returns'] = nasdaq_data['Close'].pct_change()
optimal_params = {}
best_sharpe_ratio = -np.inf

for keltner_period in range(10, 31):
    for bollinger_period in range(10, 31):
        upper_keltner, middle_keltner, lower_keltner = calculate_keltner_channel(nasdaq_data, period=keltner_period)
        upper_bollinger, middle_bollinger, lower_bollinger = calculate_bollinger_bands(nasdaq_close, period=bollinger_period)
        keltner_signal = np.where(nasdaq_close > upper_keltner, -1, np.where(nasdaq_close < lower_keltner, 1, 0))
        bollinger_signal = np.where(nasdaq_close > upper_bollinger, -1, np.where(nasdaq_close < lower_bollinger, 1, 0))
        combined_signal = keltner_signal + bollinger_signal
        sharpe_ratio = evaluate_strategy(nasdaq_data['Returns'], combined_signal)

        if sharpe_ratio > best_sharpe_ratio:
            best_sharpe_ratio = sharpe_ratio
            optimal_params['keltner_period'] = keltner_period
            optimal_params['bollinger_period'] = bollinger_period

if 'keltner_period' in optimal_params:
    print("Optimal Parameters:")
    print(f"Keltner Period: {optimal_params['keltner_period']}")
    print(f"Bollinger Period: {optimal_params['bollinger_period']}\n")

    nasdaq_close = nasdaq_data['Close']
    upper_keltner, middle_keltner, lower_keltner = calculate_keltner_channel(nasdaq_data, period=optimal_params['keltner_period'])
    upper_bollinger, middle_bollinger, lower_bollinger = calculate_bollinger_bands(nasdaq_close, period=optimal_params['bollinger_period'])
    keltner_signal = np.where(nse_close > upper_keltner, -1, np.where(nse_close < lower_keltner, 1, 0))
    bollinger_signal = np.where(nse_close > upper_bollinger, -1, np.where(nse_close < lower_bollinger, 1, 0))
    combined_signal = keltner_signal + bollinger_signal

    # Plotting signals
    plt.figure(figsize=(12, 6))
    plt.plot(nasdaq_close.index, nasdaq_close, label='NASDAQ Index')
    plt.plot(nasdaq_close.index, upper_keltner, 'r--', label='Upper Keltner Channel')
    plt.plot(nasdaq_close.index, middle_keltner, 'g--', label='Middle Keltner Channel')
    plt.plot(nasdaq_close.index, lower_keltner, 'b--', label='Lower Keltner Channel')
    plt.plot(nasdaq_close.index, upper_bollinger, 'r:', label='Upper Bollinger Band')
    plt.plot(nasdaq_close.index, middle_bollinger, 'g:', label='Middle Bollinger Band')
    plt.plot(nasdaq_close.index, lower_bollinger, 'b:', label='Lower Bollinger Band')
    plt.legend()
    plt.title('NASDAQ Index with Keltner Channel and Bollinger Bands')
    plt.xlabel('Date')
    plt.ylabel('Closing Price')
    plt.show()
else:
    print("No optimal parameters found.")



The correlation coefficient between NASDAQ and NSE indices is 0.96.
The relationship is positive with a strength of 0.96.

The lead-lag relationship between the indices is: NSE leads NASDAQ.

No optimal parameters found.
